# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [145]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [146]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [147]:
#your code here
spaceship.dropna(inplace=True)
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [148]:
y = spaceship['Transported']

In [157]:
X_num = spaceship.select_dtypes('number')
standarizer = StandardScaler().fit(X_num)
x_standarized = standarizer.transform(X_num)
df_standarizado = pd.DataFrame(x_standarized, columns=X_num.columns)

df_standarizado

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.695413,-0.345756,-0.285355,-0.309494,-0.273759,-0.269534
1,-0.336769,-0.176748,-0.279993,-0.266112,0.206165,-0.230494
2,2.002842,-0.279083,1.845163,-0.309494,5.596357,-0.226058
3,0.282540,-0.345756,0.479034,0.334285,2.636384,-0.098291
4,-0.887266,0.124056,-0.243650,-0.047470,0.220152,-0.267759
...,...,...,...,...,...,...
6601,0.833037,-0.345756,3.777285,-0.309494,1.162518,-0.203876
6602,-0.749641,-0.345756,-0.285355,-0.309494,-0.273759,-0.269534
6603,-0.199145,-0.345756,-0.285355,2.938900,-0.272885,-0.269534
6604,0.213728,-0.345756,0.339621,-0.309494,0.034826,2.600774


In [150]:
X_cat1 = spaceship.select_dtypes('object')
X_cat1.drop(columns=['PassengerId','Name','Cabin'], inplace=True)
X_cat1

,HomePlanet,CryoSleep,Destination,VIP
0,Europa,False,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,False,TRAPPIST-1e,True
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False
...,...,...,...,...
8688,Europa,False,55 Cancri e,True
8689,Earth,True,PSO J318.5-22,False
8690,Earth,False,TRAPPIST-1e,False
8691,Europa,False,55 Cancri e,False


In [151]:
X_cat2 = spaceship['Cabin']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_cat2['Cabin'] = le.fit_transform(spaceship['Cabin'])
X_cat2['Cabin']


C:\Users\FX516\AppData\Local\Temp\ipykernel_18112\3439602059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat2['Cabin'] = le.fit_transform(spaceship['Cabin'])


array([ 137, 1823,    1, ..., 4298, 1778, 1778])

In [152]:
X_cat1

,HomePlanet,CryoSleep,Destination,VIP
0,Europa,False,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,False,TRAPPIST-1e,True
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False
...,...,...,...,...
8688,Europa,False,55 Cancri e,True
8689,Earth,True,PSO J318.5-22,False
8690,Earth,False,TRAPPIST-1e,False
8691,Europa,False,55 Cancri e,False


In [153]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoded = encoder.fit_transform(X_cat1).toarray()
unique_values = [encoder.categories_[i] for i in range(len(X_cat1.columns))]
onehot_encoded = pd.DataFrame(encoded, columns=[f"{col}_{value}" for col, values in zip(X_cat1.columns, unique_values) for value in values]) 
onehot_encoded


,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6601,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6602,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6603,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6604,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [159]:
X_cat = pd.concat([pd.DataFrame(X_cat2['Cabin']), onehot_encoded], axis=1)
X = pd.concat([X_cat, df_standarizado], axis=1)
X['Cabin'] = X[0]
X.drop(columns=0, inplace=True)
X

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.695413,-0.345756,-0.285355,-0.309494,-0.273759,-0.269534,137
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.336769,-0.176748,-0.279993,-0.266112,0.206165,-0.230494,1823
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.002842,-0.279083,1.845163,-0.309494,5.596357,-0.226058,1
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.282540,-0.345756,0.479034,0.334285,2.636384,-0.098291,1
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.887266,0.124056,-0.243650,-0.047470,0.220152,-0.267759,1825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6601,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.833037,-0.345756,3.777285,-0.309494,1.162518,-0.203876,134
6602,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-0.749641,-0.345756,-0.285355,-0.309494,-0.273759,-0.269534,4293
6603,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.199145,-0.345756,-0.285355,2.938900,-0.272885,-0.269534,4298
6604,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.213728,-0.345756,0.339621,-0.309494,0.034826,2.600774,1778


In [160]:
y.shape, X.shape

((6606,), (6606, 17))

**Perform Train Test Split**

In [161]:
#your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [164]:
#your code here
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)
bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.279854825573011
RMSE 0.38007263357209753
R2 score 0.4221791728382803


c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [167]:
#your code here
forest = RandomForestRegressor(n_estimators=1000, random_state=0, max_depth=10, max_features='sqrt')
forest.fit(X_train, y_train)   
pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test, y_test))

MAE 0.2828471618947257
RMSE 0.37336127214195125
R2 score 0.4424054418581752


c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [169]:
#your code here
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)
gb_reg.fit(X_train, y_train)   
pred = gb_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test, y_test))

MAE 0.26922508910639426
RMSE 0.44219445847530436
R2 score 0.21785624357492928


c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [168]:
#your code here
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)
ada_reg.fit(X_train, y_train)   
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))

MAE 0.25032758206314165
RMSE 0.4216892168460169
R2 score 0.2887128175831717


c:\Users\FX516\Desktop\Codes\DatAnalist\DatAnalist\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [ ]:
#comment here
#El mejor es RandomForestRegressor, por 0.02, ya que e sun modelo muy completo